In [42]:
!pip install -q flwr torch torchvision

In [43]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import time
import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.3.1+cpu and Flower 1.9.0


In [44]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
   process = psutil.Process(os.getpid())
   print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
   print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 107.4 GB  | Proc size: 1.1 GB
GPU RAM Free: 13610MB | Used: 2558MB | Util  16% | Total 16376MB


In [45]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5, ))]
    )
    trainset = MNIST("./dataset", train=True, download=True, transform=transform)
    testset = MNIST("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

In [46]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):  # Use the passed 'epochs' variable here
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()  # Make sure to call .item() to get the scalar value
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch}: train loss {epoch_loss:.6f}, accuracy {epoch_acc:.6f}")



def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [52]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        epochs = config.get("epochs", 1)
        start_time = time.time()  # Start time measurement
        train(self.net, self.trainloader, epochs)
        training_time = time.time() - start_time  # Calculate duration
        print(f"Training time for Client {self.cid}: {training_time:.2f} seconds")
        return get_parameters(self.net), len(self.trainloader), {"training_time": training_time}



    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader).to_client()
    # return FlowerClient(cid, net, trainloader, valloader)

In [53]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.client_training_times = {}
    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(self, server_round: int, parameters: Parameters, client_manager: ClientManager):
        sample_size, min_num_clients = self.num_fit_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_num_clients)
        epochs_sc = 5
        epochs_hl = 3

        standard_config = {"lr": 0.001, "epochs": epochs_sc}
        higher_lr_config = {"lr": 0.0001, "epochs": epochs_hl}
        fit_configurations = []
        

        for client in clients:
            # Choose config based on the previous training time
            last_time = self.client_training_times.get(client.cid, 0)  # Default to 0 if no time recorded
            print(f"This is the last time {last_time}")
            


            config_to_use = standard_config if last_time < 13.8 else higher_lr_config
            fit_configurations.append((client, FitIns(parameters, config_to_use)))

        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        for client, fit_res in results:
            # Update training times for each client
            self.client_training_times[client.cid] = fit_res.metrics.get("training_time", 0)
        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated


    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evalua
        tion function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [54]:
# if DEVICE.type == "cuda":
#     # Use a single client to train the global model
#     client_resources = { "num_cpus": 2} 
# client_resources = { "num_cpus": 2} 

In [55]:
if DEVICE.type == "cuda":
    # Use a single client to train the global model
    client_resources = {"num_gpus": .25, "num_cpus": 2} 

In [56]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,

)

INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2024-06-27 18:14:07,673	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 32.0, 'memory': 69189982413.0, 'accelerator_type:RTX': 1.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'object_store_memory': 33938563891.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_gpus': 0.25, 'num_cpus': 2}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 4 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
(ClientAppActor pid=32948) [Client 9] fit, config: {'lr': 0.001, 'epochs': 5}
(ClientAppActor pid=32116) Epoch 0: train loss 0.027659, accuracy 0.727407
(ClientAppActor pid=37256) [Client 6] fit, config: {'lr': 0.001, 'epochs': 5} [repeated 3x across cluster]
(ClientAppActor pid=14860) Epoch 2: train loss 0.005127, accuracy 0.949444 [repeated 9x across cluster]
(ClientAppActor pid=32116) Training time for Client 0: 12.92 seconds
(ClientAppActor pid=32116) [Client 2] fit, config: {'lr': 0.001, 'epochs': 5}
(ClientAppActor pid=14860) Epoch 4: train loss 0.002935, accuracy 0.971111 [repeated 9x across cluster]
(ClientAppActor pid=37256) Training time for Client 6: 13.11 seconds [repeated 3x across cluster]
(ClientAppActor pid=37256) [Client 3] fit, conf

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=32948) Training time for Client 4: 13.35 seconds
(ClientAppActor pid=32948) Epoch 4: train loss 0.003390, accuracy 0.965741 [repeated 4x across cluster]
(ClientAppActor pid=32948) [Client 0] evaluate, config: {}


INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

This is the last time 13.213767290115356
This is the last time 12.319117784500122
This is the last time 12.495346307754517
This is the last time 12.358347415924072
This is the last time 12.200799465179443
This is the last time 13.473507165908813
This is the last time 12.272115230560303
This is the last time 12.430121183395386
This is the last time 12.160559892654419
This is the last time 12.57834529876709
(ClientAppActor pid=37256) [Client 1] fit, config: {'lr': 0.001, 'epochs': 5}
(ClientAppActor pid=37256) Epoch 0: train loss 0.002340, accuracy 0.976667 [repeated 3x across cluster]
(ClientAppActor pid=37256) Training time for Client 1: 13.21 seconds
(ClientAppActor pid=37256) [Client 3] evaluate, config: {} [repeated 9x across cluster]
(ClientAppActor pid=14860) [Client 3] fit, config: {'lr': 0.001, 'epochs': 5} [repeated 3x across cluster]
(ClientAppActor pid=32948) Epoch 2: train loss 0.001567, accuracy 0.984444 [repeated 9x across cluster]
(ClientAppActor pid=14860) Training time 

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=32948) [Client 9] evaluate, config: {}


INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 128.56s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.00450680190945665
INFO :      		round 2: 0.0021654044786846498
INFO :      		round 3: 0.0017244719841582993
INFO :      


History (loss, distributed):
	round 1: 0.00450680190945665
	round 2: 0.0021654044786846498
	round 3: 0.0017244719841582993

In [9]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=5)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Client {self.cid} loss {loss}")
        print(f"Client {self.cid} accuracy {accuracy}")
        
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE) #Load Model from here
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [10]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    client_resources=client_resources,
)

INFO flwr 2024-06-03 16:51:48,934 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)


INFO flwr 2024-06-03 16:51:54,997 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 25372973875.0, 'memory': 50745947751.0, 'node:127.0.0.1': 1.0, 'CPU': 32.0, 'GPU': 1.0}
INFO flwr 2024-06-03 16:51:55,001 | server.py:86 | Initializing global parameters
INFO flwr 2024-06-03 16:51:55,002 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2024-06-03 16:51:57,564 | server.py:277 | Received initial parameters from one random client
INFO flwr 2024-06-03 16:51:57,566 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-06-03 16:51:57,568 | server.py:101 | FL starting
DEBUG flwr 2024-06-03 16:51:57,570 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


 pid=1176) [Client 5] get_parameters
 pid=1176) [Client 6] fit, config: {}
 pid=26192) [Client 3] fit, config: {}
 pid=7204) [Client 4] fit, config: {}
 pid=20224) [Client 8] fit, config: {}
 pid=1176) Epoch 0: train loss 0.031406, accuracy 0.666111
 pid=1176) Epoch 1: train loss 0.008640, accuracy 0.913704
 pid=7204) Epoch 0: train loss 0.032170, accuracy 0.655741
 pid=1176) Epoch 2: train loss 0.005622, accuracy 0.942778
 pid=26192) Epoch 0: train loss 0.031131, accuracy 0.674074
 pid=20224) Epoch 0: train loss 0.032145, accuracy 0.666667
 pid=7204) Epoch 1: train loss 0.007931, accuracy 0.923519
 pid=1176) Epoch 3: train loss 0.004063, accuracy 0.960741
 pid=26192) Epoch 1: train loss 0.008260, accuracy 0.919815
 pid=20224) Epoch 1: train loss 0.009373, accuracy 0.907963
 pid=7204) Epoch 2: train loss 0.005402, accuracy 0.945000
 pid=26192) Epoch 2: train loss 0.005360, accuracy 0.947593
 pid=1176) Epoch 4: train loss 0.003442, accuracy 0.963333
 pid=1176) [Client 5] fit, config: {}

DEBUG flwr 2024-06-03 16:53:15,510 | server.py:232 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2024-06-03 16:53:15,545 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-06-03 16:53:15,547 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)


 pid=7204) Epoch 4: train loss 0.003275, accuracy 0.964630
 pid=7204) [Client 4] evaluate, config: {}
 pid=1176) [Client 7] evaluate, config: {}
 pid=26192) [Client 0] evaluate, config: {}
 pid=7204) Client 4 loss 0.006275855886439482
 pid=7204) Client 4 accuracy 0.94
 pid=20224) [Client 5] evaluate, config: {}
 pid=7204) [Client 3] evaluate, config: {}
 pid=1176) Client 7 loss 0.005566535132626693
 pid=1176) Client 7 accuracy 0.9416666666666667
 pid=1176) [Client 8] evaluate, config: {}
 pid=26192) Client 0 loss 0.006947518636782964
 pid=26192) Client 0 accuracy 0.95
 pid=26192) [Client 6] evaluate, config: {}
 pid=7204) Client 3 loss 0.0055223917216062545
 pid=7204) Client 3 accuracy 0.955
 pid=20224) Client 5 loss 0.005880148770908515
 pid=20224) Client 5 accuracy 0.95
 pid=7204) [Client 2] evaluate, config: {}
 pid=20224) [Client 9] evaluate, config: {}


...

DEBUG flwr 2024-06-03 16:55:01,435 | server.py:232 | fit_round 3 received 10 results and 0 failures
DEBUG flwr 2024-06-03 16:55:01,466 | server.py:168 | evaluate_round 3: strategy sampled 10 clients (out of 10)


 pid=7204) Epoch 4: train loss 0.000771, accuracy 0.991852
 pid=20224) Epoch 4: train loss 0.000829, accuracy 0.990556
 pid=20224) [Client 0] evaluate, config: {}
 pid=7204) [Client 2] evaluate, config: {}
 pid=20224) Client 0 loss 0.002689745311048076
 pid=20224) Client 0 accuracy 0.9766666666666667
 pid=7204) Client 2 loss 0.0013351145418710076
 pid=7204) Client 2 accuracy 0.98
 pid=20224) [Client 9] evaluate, config: {}
 pid=20224) Client 9 loss 0.0017798811106937743
 pid=20224) Client 9 accuracy 0.9766666666666667
 pid=20224) [Client 6] evaluate, config: {}
 pid=1176) [Client 3] evaluate, config: {}
 pid=26192) [Client 7] evaluate, config: {}
 pid=7204) [Client 4] evaluate, config: {}
 pid=26192) Client 7 loss 0.0015654092175342764
 pid=26192) Client 7 accuracy 0.98
 pid=7204) Client 4 loss 0.0017429726864793338
 pid=7204) Client 4 accuracy 0.9833333333333333
 pid=20224) Client 6 loss 0.0023496932164319634
 pid=20224) Client 6 accuracy 0.9783333333333334
 pid=20224) [Client 8] eval

DEBUG flwr 2024-06-03 16:55:03,392 | server.py:182 | evaluate_round 3 received 10 results and 0 failures
INFO flwr 2024-06-03 16:55:03,394 | server.py:147 | FL finished in 185.8248713999999
INFO flwr 2024-06-03 16:55:03,397 | app.py:218 | app_fit: losses_distributed [(1, 0.0059525216662635405), (2, 0.0022510250122771444), (3, 0.0019461998325896275)]
INFO flwr 2024-06-03 16:55:03,399 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-06-03 16:55:03,400 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-06-03 16:55:03,402 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-06-03 16:55:03,405 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.0059525216662635405
	round 2: 0.0022510250122771444
	round 3: 0.0019461998325896275

 pid=26192) Client 5 loss 0.00248527233673182
 pid=26192) Client 5 accuracy 0.98
 pid=7204) Client 1 loss 0.0020863496548796925
 pid=7204) Client 1 accuracy 0.98
